# Курсовой проект по рекомендательным системам

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

import warnings
warnings.filterwarnings('ignore')

In [34]:
data = pd.read_csv('../retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [35]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [36]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


###  Подбор оптимальных гиперпараметров для ALS

- Попробуем улучшить базовый вариант ALS, изменяя следующие параметры
  - regularization, iterations
  - factors
  - Вес (TF_IDF, BM25  взвешивание)
  
- Считаем метрики (Precision@5, MAP@5) для разных наборов гиперпараметров и выберете лучший набор

In [37]:
item_features = pd.read_csv('../product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [38]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [39]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

# Заведем фиктивный item_id (если юзер покупал товары не из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [41]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=1,
                                random_state=42)

model.fit(sparse_user_item,  # На вход item-user matrix
          show_progress=True)


CPU times: user 37.1 s, sys: 10.2 s, total: 47.2 s
Wall time: 6.69 s


Поскольку эта часть была у меня написана без библиотеки, а времени нет, то переделывать не стала

In [42]:
def get_recommendations(user, model, sparse_user_item, N=5):
    res = [id_to_itemid[rec] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item[userid_to_id[user]],   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=True)[0]]
    return res

In [43]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [44]:
%%time
    
result['als'] = result['user_id'].map(lambda x: get_recommendations(x, model, sparse_user_item, N=5))
result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()

CPU times: user 9.33 s, sys: 2.31 s, total: 11.6 s
Wall time: 1.57 s


0.18090107737511998

In [45]:
%%time

model01 = AlternatingLeastSquares(factors=100, 
                                regularization=0.01,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=1,
                                random_state=42)

model01.fit(sparse_user_item,  # На вход item-user matrix
          show_progress=True)


CPU times: user 39.3 s, sys: 9.74 s, total: 49.1 s
Wall time: 6.63 s


In [46]:
%%time
    
result['als01'] = result['user_id'].map(lambda x: get_recommendations(x, model01, sparse_user_item, N=5))
result.apply(lambda row: precision_at_k(row['als01'], row['actual']), axis=1).mean()

CPU times: user 9.15 s, sys: 2.34 s, total: 11.5 s
Wall time: 1.6 s


0.18285994123408167

In [47]:
%%time

model1 = AlternatingLeastSquares(factors=100, 
                                regularization=0.1,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=1,
                                random_state=42)

model1.fit(sparse_user_item,  # На вход item-user matrix
          show_progress=True)

result['als1'] = result['user_id'].map(lambda x: get_recommendations(x, model1, sparse_user_item, N=5))
result.apply(lambda row: precision_at_k(row['als1'], row['actual']), axis=1).mean()


CPU times: user 50.9 s, sys: 11.9 s, total: 1min 2s
Wall time: 8.29 s


0.18628795298726478

In [48]:
result.head(2)

,user_id,actual,als,als01,als1
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1105488, 1033142, 5569374, 979707, 995242]","[5569374, 1105488, 1033142, 995242, 979707]","[5569374, 1105488, 1033142, 965766, 995242]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1106523, 1133018, 910032, 5569327, 1022003]","[1106523, 1133018, 910032, 5569327, 1022003]","[1106523, 1133018, 910032, 5569327, 1022003]"


In [49]:
%%time

model_it5 = AlternatingLeastSquares(factors=100, 
                                regularization=0.1,
                                iterations=5, 
                                calculate_training_loss=True, 
                                num_threads=1,
                                random_state=42)

model_it5.fit(sparse_user_item,  # На вход item-user matrix
          show_progress=True)

result['als_it5'] = result['user_id'].map(lambda x: get_recommendations(x, model_it5, sparse_user_item, N=5))
result.apply(lambda row: precision_at_k(row['als_it5'], row['actual']), axis=1).mean()


CPU times: user 22.3 s, sys: 5.58 s, total: 27.9 s
Wall time: 3.83 s


0.19755142017629504

In [50]:
%%time

model_it25 = AlternatingLeastSquares(factors=100, 
                                regularization=0.1,
                                iterations=25, 
                                calculate_training_loss=True, 
                                num_threads=1,
                                random_state=42)

model_it5.fit(sparse_user_item,  # На вход item-user matrix
          show_progress=True)

result['als_it25'] = result['user_id'].map(lambda x: get_recommendations(x, model_it5, sparse_user_item, N=5))
result.apply(lambda row: precision_at_k(row['als_it25'], row['actual']), axis=1).mean()


CPU times: user 23.9 s, sys: 5.6 s, total: 29.5 s
Wall time: 3.9 s


0.18756121449559

In [51]:
%%time

model_fact50 = AlternatingLeastSquares(factors=50, 
                                regularization=0.1,
                                iterations=5, 
                                calculate_training_loss=True, 
                                num_threads=1,
                                random_state=42)

model_fact50.fit(sparse_user_item,  # На вход item-user matrix
          show_progress=True)

result['als_fact50'] = result['user_id'].map(lambda x: get_recommendations(x, model_fact50, sparse_user_item, N=5))
result.apply(lambda row: precision_at_k(row['als_fact50'], row['actual']), axis=1).mean()


CPU times: user 17.6 s, sys: 4.63 s, total: 22.3 s
Wall time: 3.07 s


0.19637610186091786

In [52]:
%%time

model_fact150 = AlternatingLeastSquares(factors=150, 
                                regularization=0.1,
                                iterations=5, 
                                calculate_training_loss=True, 
                                num_threads=1,
                                random_state=42)

model_fact150.fit(sparse_user_item,  # На вход item-user matrix
          show_progress=True)

result['als_fact150'] = result['user_id'].map(lambda x: get_recommendations(x, model_fact150, sparse_user_item, N=5))
result.apply(lambda row: precision_at_k(row['als_fact150'], row['actual']), axis=1).mean()


CPU times: user 31.3 s, sys: 7 s, total: 38.3 s
Wall time: 5.41 s


0.1919686581782541

Лучшая 0.19755142017629504 для
model_it5 = AlternatingLeastSquares(factors=100, 
                                regularization=0.1,
                                iterations=5, 
                                calculate_training_loss=True, 
                                num_threads=1,
                                random_state=42)                               

In [53]:
%%time

tfidf_user_item_matrix = tfidf_weight(user_item_matrix).tocsr() # Применяется к item-user матрице ! 

model_it5.fit(tfidf_user_item_matrix,  # На вход item-user matrix
          show_progress=True)

result['als_tfidf'] = result['user_id'].map(lambda x: get_recommendations(x, model_it5, sparse_user_item, N=5))

result.apply(lambda row: precision_at_k(row['als_tfidf'], row['actual']), axis=1).mean()


CPU times: user 22.7 s, sys: 5.79 s, total: 28.5 s
Wall time: 4.3 s


0.24358472086189709

In [54]:
%%time

bm25_user_item_matrix = bm25_weight(user_item_matrix.T).T.tocsr() # Применяется к item-user матрице ! 

model_it5.fit(csr_matrix(bm25_user_item_matrix).tocsr(),  # На вход item-user matrix
          show_progress=True)

result['als_bm25'] = result['user_id'].map(lambda x: get_recommendations(x, model_it5, sparse_user_item, N=5))

result.apply(lambda row: precision_at_k(row['als_bm25'], row['actual']), axis=1).mean()


CPU times: user 24.7 s, sys: 5.6 s, total: 30.3 s
Wall time: 4.24 s


0.23026444662095663

Лучшая 0.24358472086189709 для tfidf для model_it5 = AlternatingLeastSquares(factors=100, regularization=0.1, iterations=5, calculate_training_loss=True, num_threads=1, random_state=42). Эта модель дает показатель лучше, чем в бейзлайне, ориентируемся на нее

In [55]:
data_test_1 = pd.read_csv('retail_test.csv')
data_test_1

,user_id,item_id
0,1340,912987
1,588,1024426
2,2070,995242
3,1602,827939
4,1602,927712
...,...,...
79277,98,942088
79278,98,1091858
79279,98,993617
79280,98,9526886


In [57]:
result_1 = data_test_1.groupby('user_id')['item_id'].unique().reset_index()
result_1.columns=['user_id', 'actual']
result_1.head(2)

,user_id,actual
0,1,"[880007, 883616, 931136, 938004, 940947, 94726..."
1,2,"[820165, 820291, 826784, 826835, 829009, 85784..."


In [59]:
result_1['als_tfidf'] = result['user_id'].map(lambda x: get_recommendations(x, model_it5, sparse_user_item, N=5))
result_1

,user_id,actual,als_tfidf
0,1,"[880007, 883616, 931136, 938004, 940947, 94726...","[1082185, 995242, 5569374, 1100972, 965766]"
1,2,"[820165, 820291, 826784, 826835, 829009, 85784...","[1133018, 1106523, 1053690, 951590, 883404]"
2,3,"[827683, 908531, 989069, 1071377, 1080155, 109...","[1051516, 1082185, 878996, 1007195, 1024306]"
3,6,"[956902, 960791, 1037863, 1119051, 1137688, 84...","[1082185, 1058997, 6944571, 828867, 1126899]"
4,7,"[847270, 855557, 859987, 863407, 895454, 90663...","[1133018, 840361, 1053690, 1068719, 911215]"
...,...,...,...
1880,2496,"[829291, 862139, 912704, 933067, 933835, 95537...","[981760, 923746, 871756, 998556, 1092937]"
1881,2497,[6534178],"[861272, 1082185, 1026118, 995785, 999104]"
1882,2498,"[1053690, 1076875, 12386123, 858303, 920109, 1...","[904360, 1082185, 1055646, 847982, 981760]"
1883,2499,"[826249, 895327, 9858944, 820321, 829291, 8323...","[8090521, 883404, 826249, 1044078, 8090509]"


In [67]:
result_1 = result_1.drop(['actual'], axis=1)
result_1

,user_id,als_tfidf
0,1,"[1082185, 995242, 5569374, 1100972, 965766]"
1,2,"[1133018, 1106523, 1053690, 951590, 883404]"
2,3,"[1051516, 1082185, 878996, 1007195, 1024306]"
3,6,"[1082185, 1058997, 6944571, 828867, 1126899]"
4,7,"[1133018, 840361, 1053690, 1068719, 911215]"
...,...,...
1880,2496,"[981760, 923746, 871756, 998556, 1092937]"
1881,2497,"[861272, 1082185, 1026118, 995785, 999104]"
1882,2498,"[904360, 1082185, 1055646, 847982, 981760]"
1883,2499,"[8090521, 883404, 826249, 1044078, 8090509]"


In [70]:
result_1.to_csv("test_users.csv", index=False)